In [18]:
import pandas as pd
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', '..', '03-src')
sys.path.append(src_dir)

import decisionclass.decision_functions as hmd

from sklearn.preprocessing import MinMaxScaler

# from itertools import combinations
%matplotlib inline

In [19]:
cereal_df = pd.read_csv('../../01-data/01-raw/grocery-store/cereal.csv')

In [20]:
cereal_df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [21]:
cereal_df.describe()

,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,106.883117,2.545455,1.012987,159.675325,2.151948,14.597403,6.922078,96.077922,28.246753,2.207792,1.029610,0.821039,42.665705
std,19.484119,1.094790,1.006473,83.832295,2.383364,4.278956,4.444885,71.286813,22.342523,0.832524,0.150477,0.232716,14.047289
min,50.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,1.000000,0.500000,0.250000,18.042851
25%,100.000000,2.000000,0.000000,130.000000,1.000000,12.000000,3.000000,40.000000,25.000000,1.000000,1.000000,0.670000,33.174094
50%,110.000000,3.000000,1.000000,180.000000,2.000000,14.000000,7.000000,90.000000,25.000000,2.000000,1.000000,0.750000,40.400208
75%,110.000000,3.000000,2.000000,210.000000,3.000000,17.000000,11.000000,120.000000,25.000000,3.000000,1.000000,1.000000,50.828392
max,160.000000,6.000000,5.000000,320.000000,14.000000,23.000000,15.000000,330.000000,100.000000,3.000000,1.500000,1.500000,93.704912


---
---

## Clean the data

Can we interpret the null values (-1)?

In [22]:
cereal_df.loc[(cereal_df.carbo==-1) | (cereal_df.sugars==-1) | (cereal_df.potass==-1)]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843
20,Cream of Wheat (Quick),N,H,100,3,0,80,1.0,21.0,0,-1,0,2,1.0,1.00,64.533816
57,Quaker Oatmeal,Q,H,100,5,2,0,2.7,-1.0,-1,110,0,1,1.0,0.67,50.828392


Cream of Wheat nutrition information found online [here](https://www.creamofwheat.com/product/original).

In [23]:
cereal_df.at[20, 'potass'] = 0

Quaker Oatmeal nutrition information found online [here](https://www.nutritionix.com/i/quaker-oats/old-fashioned-oats/51c54a5d97c3e6efadd60584). 
Values are scaled to 2/3 of 1 cup.

In [24]:
cereal_df.at[57, 'carbo'] = 18
cereal_df.at[57, 'sugars'] = 1

Unable to determine Almond Delight nutrition information (or even what exactly Almond Delight is!). 
We will drop this row.

In [25]:
cereal_df = cereal_df.drop([4])

In [26]:
cereal_df.loc[(cereal_df.carbo==-1) | (cereal_df.sugars==-1) | (cereal_df.potass==-1)]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating


---
### Clean the data

Are ```carbo``` $>$ ```sugar``` $+$ ```fiber```?

In [27]:
cereal_df.loc[cereal_df.carbo < cereal_df.sugars+cereal_df.fiber]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.00,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.00,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.00,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.00,0.50,93.704912
5,Apple Cinnamon Cheerios,G,C,110,2,2,180,1.5,10.5,10,70,25,1,1.00,0.75,29.509541
6,Apple Jacks,K,C,110,2,0,125,1.0,11.0,14,30,25,2,1.00,1.00,33.174094
14,Cocoa Puffs,G,C,110,1,1,180,0.0,12.0,13,55,25,2,1.00,1.00,22.736446
18,Count Chocula,G,C,110,1,1,180,0.0,12.0,13,65,25,2,1.00,1.00,22.396513
19,Cracklin' Oat Bran,K,C,110,3,3,140,4.0,10.0,7,160,25,3,1.00,0.50,40.448772
22,Crispy Wheat & Raisins,G,C,100,2,1,140,2.0,11.0,10,120,25,3,1.00,0.75,36.176196


In [28]:
def carbo_replace(df_row):
    """
    Replaces carbo with sum of sugar and fiber if that value is higher
    
    Parameters
    ----------
    df_row: Series
        row of a DataFrame. Expectes columns named 'sugars', 'fiber' and 'carbo'
        
    Returns
    -------
    calculated_carbo: float
        Calories calculated using 9*fat+4(carbs+protein) where fat, carbs, and protein are in grams
    or
    df_row.carbo: float
        The given calorie count
    """
    calculated_carbo = df_row.sugars + df_row.fiber
    if df_row.carbo<calculated_carbo:
        return calculated_carbo
    else:
        return df_row.carbo

In [29]:
cereal_df.carbo = cereal_df.apply(carbo_replace, axis=1)
cereal_df.loc[cereal_df.carbo < cereal_df.sugars+cereal_df.fiber]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating


---

### Clean the data:

Are ```calories``` greater than $9\cdot$ ```fat``` $+4\cdot($ ```carbo``` $+$ ```protein``` $)$? (Note that the summary statistics show that equality won't hold always.)

In [30]:
cereal_df.loc[cereal_df.calories<9*cereal_df.fat+4*cereal_df.carbo+4*cereal_df.protein]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,16.0,6,280,25,3,1.0,0.33,68.402973
2,All-Bran,K,C,70,4,1,260,9.0,14.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,14.0,0,330,25,3,1.0,0.50,93.704912
54,Puffed Rice,Q,C,50,1,0,0,0.0,13.0,0,15,0,3,0.5,1.00,60.756112
57,Quaker Oatmeal,Q,H,100,5,2,0,2.7,18.0,1,110,0,1,1.0,0.67,50.828392
65,Shredded Wheat spoon size,N,C,90,3,0,0,3.0,20.0,0,120,0,1,1.0,0.67,72.801787


None of these values are dramatic discrepencies. 
We will change the total calorie count to be $9\cdot$ ```fat``` $+4\cdot($ ```carbo``` $+$ ```protein``` $)$.

In [31]:
def calorie_replace(df_row):
    """
    Replaces calories with calculated calories if that value is higher
    
    Parameters
    ----------
    df_row: Series
        row of a DataFrame. Expectes columns named 'fat', 'carbo', and 'protein'
        
    Returns
    -------
    calculated_calories: float
        Calories calculated using 9*fat+4(carbs+protein) where fat, carbs, and protein are in grams
    or
    df_row.calories: float
        The given calorie count
    """
    calculated_calories = 9*df_row.fat + 4*df_row.carbo + 4*df_row.protein
    if df_row.calories<calculated_calories:
        return calculated_calories
    else:
        return df_row.calories

In [32]:
cereal_df.calories = cereal_df.apply(calorie_replace, axis=1)
cereal_df.loc[cereal_df.calories<9*cereal_df.fat+4*cereal_df.carbo+4*cereal_df.protein]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating


---
---

### Feature Engineering

* Net Carbs is a thing people track, lets make a column for that.
* Make a calories per cup ratio column
* Make carbo/calories, protein/calories, fat/calories, sugars/calories, fiber/calories, & sodium/calories

In [33]:
cereal_df['net_carbs'] = cereal_df.carbo - cereal_df.fiber
cereal_df['cal_per_cup'] = cereal_df.calories/cereal_df.cups
cereal_df['carbo_per_cal'] = cereal_df.carbo/cereal_df.calories
cereal_df['protein_per_cal'] = cereal_df.protein/cereal_df.calories
cereal_df['fat_per_cal'] = cereal_df.fat/cereal_df.calories
cereal_df['sugars_per_cal'] = cereal_df.sugars/cereal_df.calories
cereal_df['fiber_per_cal'] = cereal_df.fiber/cereal_df.calories
cereal_df['sodium_per_cal'] = cereal_df.sodium/cereal_df.calories

### Scale Features out of 10

First, remove columns ```mfr```, ```type```, ```vitamins```, ```shelf```, ```weight```, & ```rating```.

In [ ]:
clean_cereal_df = cereal_df.drop(['mfr', 'type', 'vitamins', 'shelf', 'weight', 'rating'], axis=1)

In [50]:
scaler = MinMaxScaler(feature_range=(0,10))
scaled_features = scaler.fit_transform(clean_cereal_df.drop('name', axis=1))

In [51]:
feature_columns = clean_cereal_df.drop('name', axis=1).columns.values
features_df = pd.DataFrame(scaled_features, columns=feature_columns)
df = pd.merge(clean_cereal_df[['name']], features_df, left_index=True, right_index=True)

In [52]:
df.describe()

,calories,protein,fat,sodium,fiber,carbo,sugars,potass,cups,net_carbs,cal_per_cup,carbo_per_cal,protein_per_cal,fat_per_cal,sugars_per_cal,fiber_per_cal,sodium_per_cal
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,5.248485,3.120000,1.973333,4.964583,1.554286,4.487179,4.595556,2.961616,4.583467,5.937971,2.448870,4.510558,3.401903,2.055331,4.092893,1.118617,4.551451
std,1.646728,2.211518,2.013289,2.649730,1.721664,2.459762,2.964400,2.159232,1.884343,1.781135,1.604058,2.409528,2.396582,2.003657,2.537982,1.500402,2.543845
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.545455,2.000000,0.000000,3.984375,0.357143,2.307692,2.000000,1.212121,3.360000,5.217391,1.538462,2.402909,2.085561,0.000000,1.818182,0.214286,3.327797
50%,5.454545,4.000000,2.000000,5.312500,1.428571,3.846154,4.000000,2.727273,4.000000,5.652174,2.162266,4.174545,2.941176,2.181818,4.242424,0.857143,4.895604
75%,5.454545,4.000000,2.000000,6.718750,2.142857,6.153846,7.333333,3.636364,6.000000,6.521739,2.874091,6.247857,4.588235,2.829796,6.085859,1.542857,6.230769
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [53]:
df.to_pickle('../../01-data/02-decision-basis/cereal_decision.pkl')